In [1]:
from typing import Dict, Tuple

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 100)


In [ ]:
# notebook goal: Setup a basic machine learning framework that cleans data, standardizes features,
#  evaluates feature impt, shap values, and a myriad of ML algorithms
# TODO: add the day-of-week as a feature
# TODO: Add in target date versus historic reference dates
# TODO: Add in volume-based feature functionality
# TODO: Evaluate standardizing features per stock or one model per stock - may not be enough data realistically
# TODO: Check bol-range-pct calculation - only giving zero value

In [95]:
# functions:

def clean_stock_data(dataframe: pd.DataFrame) -> pd.DataFrame :

    '''removes nulls and in the future will be built out to do any additonal cleaning on the dataframe that is necessary
    Args:
        dataframe: pandas dataframe containing all of the potential features
        parameters: 
            calculation_field: field on which all of the features are built

    Returns:
        dataframe: dataset that is ready to load into a machine learning framework
    '''

    #TODO: In pipeline write this output to the 
    # remove records the preceed the target period to have complete information:
    dataframe.dropna(inplace = True)
    #dataframe = dataframe.reset_index(drop = True) # we won't reset the index for now for traceability back to the date, ticker combination later after training

    # set the date as an index to us post-forecasting: This is a bad idea, come back to the concept
    #dataframe.set_index(keys = 'date', verify_integrity = False, inplace = True) # verify integrity Fale to allow duplicates**
    
    # remove fields that will not be used as predictive features (can be hardcoded since dataframe structure will be the same):
    dataframe = dataframe.drop(columns = [ 'date', 'high', 'low', 'open', 'volume', 'adj_close'])
    

    return dataframe


def identify_fields_to_standardize(dataframe: pd.DataFrame, parameters: Dict) -> np.array :

    '''creates a list of the continuous fields to standardize by dimension within the predictive model; NOTE: this is used within the standardizer
    
    Args:
        dataframe: dataframe that contains all of the fields of interest to be used in the calculations
        parameters:
            continuous_feature_cutoff: ratio of unique values to record count to be used to codify continuous features -> removes records from the standardization process which don't have enough data to standardize (e.g., boolean)

    Returns: list of continuous fields to use in the standardization process based on user's specifications of "uniqueness" threshold    

    '''

    numeric_fields = dataframe.select_dtypes(include = 'number').columns
    records = len(dataframe)

    record_summary = pd.DataFrame(dataframe[numeric_fields].nunique(), columns = ['unique_values'])
    record_summary['rows_in_df'] = records
    record_summary['value_to_record_ratio'] = record_summary['unique_values']/ record_summary['rows_in_df']

    # filter for a threshold specified by the user:
    record_summary = record_summary[record_summary['value_to_record_ratio'] > parameters['continuous_feature_cutoff']]

    # remove percentage features # TODO: later add in functionality to remove percentage based features

    return record_summary.index


# Justification for approach on scaling - the argument can be made that since our approach will generalize movemements across multiple securities that we need to standardize each security to its own price range.  Therefore, any features with price-relative values will be scaled per the security's price values to avoid odd splits in tree-based algos
# the concern with standardization is generally focused on not letting any one feature have considerably more weight in a model than another; however in this case, 


def standardize_continuous_features(dataframe: pd.DataFrame, parameters: Dict) -> pd.DataFrame:

    '''function that identifies the continuious features in the dataframe and standardizes each feature by equity to enable scaling relative to each equity
    
    Args:
        Dataframe: Pandas dataframe to be used in machine learning
        Parameters:
            stock_field: field indicating the stock for the window function to scan
            calculation_field: field for which the target is being calculated (used for drop in main row merge)
    
    Returns:
        Dataframe: containing the standardized data fields
    
    '''

    continuous_fields = list(identify_fields_to_standardize(dataframe = dataframe, parameters = parameters))

    # add in the ticker for grouping next:
    continuous_fields.append(parameters['stock_field'])

    # downselect to the fields that will be used to standardize:
    continuous_dataframe = dataframe[continuous_fields]

    # calculate z-scores: --> Standardizes within each feature to scale accordingly
    z_scores = (continuous_dataframe - continuous_dataframe.groupby(by = parameters['stock_field']).transform('mean')) / continuous_dataframe.groupby(by = parameters['stock_field']).transform('std')

    # drop the null ticker (not needed post groupby): 
    z_scores.drop(columns = [ parameters['stock_field'], parameters['calculation_field'] ], inplace = True)

    # rename the fields to indicate standardization:
    z_scores.columns = z_scores.columns + '_std'

    # drop original continuous fields # TODO: coming back after calculation checks:
    if parameters['drop_original_fields'] == True:
        continuous_fields.remove(parameters['stock_field'])
        dataframe.drop(columns = continuous_fields, inplace = True)

    # append the fields back into the core dataframe:
    z_scores = pd.concat([dataframe, z_scores], axis = 1)

    # remove the standardized target field:
    z_scores.drop(columns = z_scores.columns[z_scores.columns.str.contains('target')][1], inplace = True)

    # remove unnecessary items:
    del continuous_fields, continuous_dataframe

    return z_scores



def one_hot_encode_tickers(dataframe: pd.DataFrame, parameters: Dict) -> pd.DataFrame:

    '''Returns one-hot encoded features to the predictive dataset NOTE: May not work, but this retains some of the information in the original dataframe while also potentially giving the global model a nudge
       Note: we choose not to drop first for now, even though it's a trap; Can be used post processing or as model features
    Args:
        dataframe: core dataset that has been augmented with additional features
        parameters:
            stock_field: text field containing the 
    Returns:   
        dataframe with augmented columns
    
    '''

    dataframe = pd.get_dummies(data = dataframe, prefix = "ind", columns = [parameters['stock_field']], drop_first = False)

    return dataframe



def create_training_test_splits(dataframe: pd.DataFrame, parameters: Dict) -> Tuple:

    '''Function that splits out training and test sets for machine learning; for the purposes of this model the way we piose the problem allows for random train test split
    Args:
        dataframe: pandas dataframe containing only the target field and the features to be used by the classifier
        parameters:
            test_ratio: proportion of samples in the dataframe to be used as a test set once the models are tuned and evaluated

    '''

    # define Y and x:
    target_feature = list(dataframe.columns[dataframe.columns.str.contains('target')])

    y = dataframe[target_feature]
    X = dataframe.drop(columns = target_feature)

    # create the training and test splits:
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=parameters['test_size'], random_state=parameters['seed'])

    return X_train, X_test, y_train, y_test




In [96]:
df = catalog.load('combined_modeling_input')

[06/11/23 11:21:30] INFO     Loading data from 'combined_modeling_input' (CSVDataSet)...        ]8;id=44944;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=792196;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/kedro/io/data_catalog.py#344\344]8;;\

In [97]:
# test: clean stock data:

df = clean_stock_data(dataframe = df)

In [98]:
parameters = {'continuous_feature_cutoff' : 0.6,
              'stock_field' : 'ticker',
              'calculation_field' : 'close',
              'drop_original_fields' : True,
              'drop_stock_field': True, # keep this fixed 
              'train_size' : 0.20,
              'seed' : 1187
              }

In [99]:
# test: standardize features:
test = standardize_continuous_features(dataframe = df, parameters = parameters)




In [117]:
# one-hot encode: 
test = one_hot_encode_tickers(dataframe = test, parameters= parameters)

In [ ]:
##################################### - Function development HERE

In [115]:
test_1 = one_hot_encode_tickers(dataframe= test, parameters = parameters)

In [118]:
test.head()

,above_7_close_sma_ind,above_14_close_sma_ind,above_21_close_sma_ind,cum_days_above_above_7_close_sma_ind,cum_days_above_above_14_close_sma_ind,cum_days_above_above_21_close_sma_ind,bol_range_pct,target_20_days_ahead_ind,14_close_sma_std,14_close_sma_pct_diff_std,14_close_std_std,21_close_sma_std,21_close_sma_pct_diff_std,21_close_std_std,7_close_sma_std,7_close_sma_pct_diff_std,7_close_std_std,bol_pct_from_bottom_std,bol_pct_from_top_std,bol_range_std,lower_bollinger_band_std,upper_bollinger_band_std,ind_AAPL,ind_XLE,ind_XLF
0,1.0,1.0,1.0,2.0,8.0,1.0,0.0,1,-1.595435,-1.376604,-0.986506,-1.593107,-1.215422,-1.125114,-1.595039,-1.734311,-0.509108,1.030173,1.116424,-1.125114,-1.577246,-1.598994,1,0,0
1,1.0,1.0,1.0,3.0,9.0,2.0,0.0,1,-1.589468,-1.232396,-0.897239,-1.590706,-1.174615,-1.051856,-1.584461,-1.340545,-0.435440,1.137107,0.938561,-1.051856,-1.582146,-1.590141,1,0,0
2,1.0,1.0,1.0,4.0,10.0,3.0,0.0,1,-1.580557,-1.670412,-0.782983,-1.582594,-1.530326,-1.009586,-1.568888,-1.761056,-0.374703,1.557244,1.241071,-1.009586,-1.577831,-1.578709,1,0,0
3,1.0,1.0,1.0,5.0,11.0,4.0,0.0,1,-1.571705,-1.846688,-0.635644,-1.575358,-1.697122,-0.900143,-1.555102,-1.856323,-0.221543,1.943526,1.225300,-0.900143,-1.581276,-1.562054,1,0,0
4,1.0,1.0,1.0,6.0,12.0,5.0,0.0,0,-1.563611,-1.652952,-0.530130,-1.568013,-1.557752,-0.829720,-1.540041,-1.309295,-0.291924,1.936549,0.962560,-0.829720,-1.580639,-1.548809,1,0,0


In [212]:
##################################### - Testing functions HERE

In [128]:
def create_training_test_splits(dataframe: pd.DataFrame, parameters: Dict) -> Tuple:

    '''Function that splits out training and test sets for machine learning; for the purposes of this model the way we piose the problem allows for random train test split
    Args:
        dataframe: pandas dataframe containing only the target field and the features to be used by the classifier
        parameters:
            test_ratio: proportion of samples in the dataframe to be used as a test set once the models are tuned and evaluated

    '''

    # define Y and x:
    target_feature = list(dataframe.columns[dataframe.columns.str.contains('target')])

    y = dataframe[target_feature]
    X = dataframe.drop(columns = target_feature)

    # create the training and test splits:
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=parameters['test_size'], random_state=parameters['seed'])

    return X_train, X_test, y_train, y_test

In [129]:
create_training_test_splits(dataframe = test, parameters  = parameters)

['target_20_days_ahead_ind']

In [232]:
# classifiers to use: support vactor machine, decision tree, random forest, xgboost, adaboost

def train_models(X_train: pd.DataFrame, y_train: pd.Series, parameters) -> pd.DataFrame:

    '''Trains a series of machine learning model outputs for evaluation by the user
    
    Args:
        X_train: inputs from train-test split function
        y_train: y-series from the train-test split function

    Returns:
        Summarized output of all ML models tried
    
    '''


Index(['date', 'ticker', 'above_7_close_sma_ind', 'above_14_close_sma_ind',
       'above_21_close_sma_ind', 'cum_days_above_above_7_close_sma_ind',
       'cum_days_above_above_14_close_sma_ind',
       'cum_days_above_above_21_close_sma_ind', 'bol_range_pct',
       'target_20_days_ahead_ind', '14_close_sma_std',
       '14_close_sma_pct_diff_std', '14_close_std_std', '21_close_sma_std',
       '21_close_sma_pct_diff_std', '21_close_std_std', '7_close_sma_std',
       '7_close_sma_pct_diff_std', '7_close_std_std',
       'bol_pct_from_bottom_std', 'bol_pct_from_top_std', 'bol_range_std',
       'lower_bollinger_band_std', 'upper_bollinger_band_std'],
      dtype='object')